In [1]:
# Jan 29, 2020
# many errors appeared after joining the dataset 
# i rewritten code into simpler scripts
# this script joins variables together to be processed later

# they are:
#     - cit_delay - reads clean_patent and clean_uspatentcitation and calculates delay by patent
#     - cit_made - reads clean_uspatentcitation and calculates citations made 
#     - cit_received - reads clean_uspatentcitation and calculates citations received
#     - cit_tree - reads clean_uspatentcitation and cit_made and calculates parent_citation
#     - generalit - reads wipo and clean_uspatentcitation and calculates generality
#     - originality - reads wipo and clean_uspatentcitation and calculates originality
#     - wipo_first_class - reads wipo and generates wipo_first_class

# it is now way faster, but I cannot know for sure if this is a consequence of coding or something changed in 
# infrastructure. 

# since class is a variable that also appear in non-cited patents, I'll include them in the final step


# Jan, 20th 2020
# join delay and tree data to form the analysis dataset


In [2]:
import pandas as pd
import numpy as np

In [3]:
received='data/cit_received.csv'
made='data/cit_made.csv'
received_delay='data/cit_received_delay.csv'
made_delay='data/cit_made_delay.csv'
cit_tree = 'data/cit_tree.csv'
originality = 'data/originality.csv'
generality = 'data/generality.csv'
dst='data/dataset.csv'
wipo = 'data/wipo_first_class.csv' #avoid multiple classes


In [4]:
dfs=[]

In [5]:
def data_read(file, names, usecols, dtype):
    df = pd.read_csv(file, names=names, usecols=usecols, dtype=dtype, index_col='id', header=0)
    dfs.append(df)

In [6]:
names=['index', 'id', 'cit_received']
usecols=['id', 'cit_received']
dtype={'id':object}
data_read(received, names, usecols, dtype)

In [7]:
names=['index', 'id', 'cit_made']
usecols=['id', 'cit_made']
dtype={'id':object}
data_read(made, names, usecols, dtype)

In [8]:
names=['id', 'cit_received_delay']
dtype={'id':object}
usecols=['id', 'cit_received_delay']
data_read(received_delay, names, usecols, dtype)

In [9]:
names=['id', 'cit_made_delay']
dtype={'id':object}
usecols=['id', 'cit_made_delay']
data_read(made_delay, names, usecols, dtype)

In [10]:
names=['id', 'parent_citation']
dtype={'id':object}
usecols=['id', 'parent_citation']
data_read(cit_tree, names, usecols, dtype)

In [11]:
names=['id', 'df_squared','total_citation','herfindal','originality']
dtype={'id':object}
usecols=['id', 'originality']
data_read(originality, names, usecols, dtype)

In [12]:
names=['id', 'df_squared','total_citation','herfindal','generality']
dtype={'id':object}
usecols=['id', 'generality']
data_read(generality, names, usecols, dtype)

In [20]:
%time
# dfs = [df, df_made, df_received_delay]
# dfs = [df.set_index('id') for df in dfs]
df=dfs[0].join(dfs[1:])

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs


In [21]:
df.dtypes

cit_received            int64
cit_made              float64
cit_received_delay    float64
cit_made_delay        float64
parent_citation       float64
originality           float64
generality            float64
dtype: object

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7048984 entries, 0000000 to re25727
Data columns (total 7 columns):
cit_received          int64
cit_made              float64
cit_received_delay    float64
cit_made_delay        float64
parent_citation       float64
originality           float64
generality            float64
dtypes: float64(6), int64(1)
memory usage: 430.2+ MB


In [23]:
df.sample(n=10)

,cit_received,cit_made,cit_received_delay,cit_made_delay,parent_citation,originality,generality
id,,,,,,,
20040055564,1,NaN,0.846575,NaN,3.0,NaN,0.000000
3852265,5,NaN,32.397260,NaN,NaN,NaN,0.320000
1831518,2,NaN,64.472603,NaN,13.0,NaN,0.000000
7227849,82,18.0,8.456331,7.829528,620.0,0.444444,0.566330
D30089,19,NaN,101.746271,NaN,265.0,NaN,0.444444
3026041,3,NaN,29.336986,NaN,30.0,NaN,0.444444
7769742,27,11.0,4.419990,7.562391,62.0,0.314050,0.071331
9129552,2,26.0,1.726027,15.065121,NaN,0.677515,0.000000
8328659,59,40.0,3.712840,32.517808,190.0,0.000000,0.000000


In [24]:
df.describe()

/home/applications/anaconda3/4.2.0/lib/python3.5/site-packages/numpy/lib/function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,cit_received,cit_made,cit_received_delay,cit_made_delay,parent_citation,originality,generality
count,7.048984e+06,4.712934e+06,7.018034e+06,4.509640e+06,6.256483e+06,4.195437e+06,6.552762e+06
mean,1.297282e+01,1.468080e+01,2.036854e+01,1.335864e+01,2.472962e+02,2.731046e-01,2.534382e-01
std,3.053055e+01,3.929394e+01,2.422966e+01,1.072210e+01,1.588333e+03,2.654199e-01,2.652160e-01
min,1.000000e+00,0.000000e+00,-3.514364e+01,-2.805233e+02,1.000000e+00,0.000000e+00,0.000000e+00
25%,2.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN
50%,5.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.300000e+01,NaN,NaN,NaN,NaN,NaN,NaN
max,3.922000e+03,5.811000e+03,2.075753e+02,1.827425e+02,2.274790e+05,9.391847e-01,9.235959e-01


In [25]:
# df=df.merge(wipo, left_index=True, right_index=True, how='outer')

In [26]:
# df.sample(n=10)

,cit_received,cit_made,cit_received_delay,cit_made_delay,parent_citation,originality,generality,field_id
id,,,,,,,,
3661021,20.0,NaN,25.358082,NaN,240.0,NaN,0.255000,NaN
8674332,4.0,15.0,2.640411,5.581187,1.0,0.48000,0.375000,8.0
6141294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
9915730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
6496615,4.0,6.0,2.824658,1.518265,10.0,0.32000,0.500000,9.0
2393518,1.0,NaN,71.380822,NaN,NaN,NaN,0.000000,NaN
8752365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0
9061082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0
5530293,61.0,8.0,10.874117,6.091096,915.0,0.40625,0.206396,8.0


In [27]:
# df.describe()

/home/applications/anaconda3/4.2.0/lib/python3.5/site-packages/numpy/lib/function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,cit_received,cit_made,cit_received_delay,cit_made_delay,parent_citation,originality,generality,field_id
count,7.048984e+06,4.712934e+06,7.018034e+06,4.509640e+06,6.256483e+06,4.195437e+06,6.552762e+06,6.441314e+06
mean,1.297282e+01,1.468080e+01,2.036854e+01,1.335864e+01,2.472962e+02,2.731046e-01,2.534382e-01,1.502865e+01
std,3.053055e+01,3.929394e+01,2.422966e+01,1.072210e+01,1.588333e+03,2.654199e-01,2.652160e-01,1.082388e+01
min,1.000000e+00,0.000000e+00,-3.514364e+01,-2.805233e+02,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,3.922000e+03,5.811000e+03,2.075753e+02,1.827425e+02,2.274790e+05,9.391847e-01,9.235959e-01,3.500000e+01


In [28]:
df.to_csv(dst)

In [13]:
# this code includes WIPO in the dataset
# as class is also part of non-cited patents, I'm not including in here.
# in any case, for some reason this merge could not be done by the method used in the previous
# there is a bug when using read_csv using dtype and indexing. 
# it is a known bug in the community but i do not know why the previous join did not accused the issue
# the method below corrects the issue 

# names=['index', 'id', 'field_id']
# dtype={'id':object}
# usecols=['id', 'field_id']
# # data_read(wipo, names, usecols, dtype)
# wipo = pd.read_csv(wipo, names=names, usecols=usecols, dtype=dtype, header=0)
# wipo['id']=wipo['id'].astype(str)
# wipo=wipo.set_index('id')
# wipo.info()

# dfs.append(wipo)